In [1]:
import pandas as pd
import numpy as np

#local 読み込み
dataPath = './datasets/'
titlePath = './data/titledata/'
keyPath = './data/keyworddata/'
storyPath = './data/storydata/'
allPath =  './data/all/'

test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

In [3]:
name = "sudachi"

title_mecab_data = pd.read_csv(titlePath+'sudachi.csv')
key_mecab_data = pd.read_csv(keyPath+'sudachi.csv' )
story_mecab_data = pd.read_csv(storyPath+'sudachi.csv')

data = pd.DataFrame()
data["sentence"] = title_mecab_data["sudachi"] + " " + key_mecab_data["sudachi"] + " "+ story_mecab_data["sudachi"]


In [4]:
from gensim.models import word2vec

#[[文章の単語], [文章の単語]]を作成
title_dic = []
def make_title_dic(text):
  if text is np.nan:
    return
  title_dic.append(text.split(' '))
data['sentence'].map(make_title_dic)

#学習
model = word2vec.Word2Vec(title_dic,vector_size=200, min_count=5, window=5, epochs=20)

#作成した辞書をcsvに保存して可視化(処理には関係ない)
from gensim import corpora#辞書を作るためのもの
dictionary = corpora.Dictionary(title_dic)#textsをもとに辞書を作成します
dictionary.filter_extremes(no_below=5)#出現文書数が5回以下のものはさようなら
dictionary.save_as_text(allPath + 'dictionary.csv')#辞書を保存


In [5]:
dimension = 200 #次元

#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text):
  if text is np.nan:
    return np.array([np.nan for _ in range(dimension)])
  L = []
  for w in text.split(' '):
    if w in model.wv.key_to_index:
      L.append(model.wv.get_vector(key=w))
  if len(L)==0:
    return np.array([np.nan for _ in range(dimension)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data, rowName):
  vec_list = []
  for index, row in data.iterrows():
    vec_list.append(getSentenceVector(row[rowName]))
  return np.array(vec_list)

data = pd.read_csv(titlePath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'title_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

print("title 完了")

data = pd.read_csv(storyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'story_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

print("story 完了")

data = pd.read_csv(keyPath+'sudachi.csv', index_col=0)
vec_list_array = makeVectorArrayList(data, "sudachi")
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(allPath+'key_vec_originalLearned_sudachi_'+ str(dimension) +'.csv')

title 完了
story 完了
